In [16]:
"""
This notebook contains code for drawing face-detected and random samples from a video, 
annotating on face/no face, calculating precision, recall, and F-score, 
and visualizing the scores across detectors, groups, and videos.
"""
%load_ext autoreload
%autoreload 2

import os
import sys
import subprocess
import ntpath

from collections import defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import collections  as mc

print('Importing FaceDetector...')
from config import *
from detector import FaceDetector
from utils import (format_num, create_sample_json, annotate_sample, 
                   run_detector_on_sample, incorporate_openpose_output, 
                   calc_prf, display_prf, display_prf2, submit_sbatch, run_openpose)

print(sys.path) #should somewhere include [...]/miniconda2/envs/headcam/[...]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Importing FaceDetector...
['/home/users/agrawalk/miniconda2/envs/headcam/lib/python36.zip', '/home/users/agrawalk/miniconda2/envs/headcam/lib/python3.6', '/home/users/agrawalk/miniconda2/envs/headcam/lib/python3.6/lib-dynload', '', '/home/users/agrawalk/.local/lib/python3.6/site-packages', '/home/users/agrawalk/miniconda2/envs/headcam/lib/python3.6/site-packages', '/home/users/agrawalk/miniconda2/envs/headcam/lib/python3.6/site-packages/IPython/extensions', '/home/users/agrawalk/.ipython']


In [17]:
#1. Extract frames from video

for vid_path in ['NEW_VID_PATHS']:
    msg = submit_sbatch(f'python extract_frames.py {vid_path}', job_name='extract', p='normal,hns', t=2)
    print(msg)

(b'Submitted batch job 44773571\n', b'')


In [26]:
#Confirm jobs are submitted
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          44971496       gpu demovide agrawalk PD       0:00      1 (Priority)
          44956279       hns  jupyter agrawalk  R    4:45:05      1 sh-108-15


In [ ]:
#2. Run MTCNN on 10000 frames of each video
#TODO: add case to check if output json exists, and ask for confirmation to overwrite

for frame_dir in FRAME_DIRS:
    msg = submit_sbatch(f'python detect_faces_simple.py {vid_path} {MASTER_JSON_PATH}', job_name='extract', p='normal,hns', c=8, t=2)
    print(msg)

In [ ]:
#3. select a random sample of [sample_size] face-detected, [sample_size] random frames from each video in the dataframe
#e.g. if 6 videos in JSON and sample_size=200, creates a sample dataframe of size (200 + 200)*6 = 2400 frames
#TODO: add case to check if output json exists, and ask for confirmation to overwrite

create_sample_json(MASTER_JSON_PATH, SAMPLE_JSON_PATH, sample_size=200)

In [ ]:
#4a. run + add detections for additional detectors to sample dataframe.

for det_name in ['vj']:
    run_detector_on_sample(det_name, OUTPUT, SAMPLE_JSON_PATH)

In [21]:
run_openpose(DEMO_VID_PATH, DEMO_OUTPUT, no_display=True, 
             render_pose=0, keypoint_scale=3, 
             frame_rotate=180, face=True, hand=True)

overwrite existing directory /scratch/users/agrawalk/demo/demovideo? (yes/no) yes


In [25]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          44971496       gpu demovide agrawalk PD       0:00      1 (Priority)
          44956279       hns  jupyter agrawalk  R    4:42:41      1 sh-108-15


In [ ]:
#4b. Run openpose on videos

# Old format of the command, for reference
# cmd = ('sbatch -p gpu --gres gpu:1 -t 5:00:00 --mem 8G '
#        '--mail-type=FAIL --mail-user=agrawalk@stanford.edu '
#        '--wrap="singularity exec --nv $SINGULARITY_CACHEDIR/openpose-latest.img bash -c '
#        '\'cd /openpose-master && ./build/examples/openpose/openpose.bin '
#        '--no_display true '
#        '--render_pose 0 '
#        '--video {0} '
#        '--keypoint_scale 3 '
#        '--frame_rotate 180 '
#        '--face ' # maybe don't want this
#        '--hand ' # probably don't want this
#        '--write_keypoint_json {1}\'"')

for vid_path in NEW_VID_PATHS:
    msg = run_openpose(vid_path, OPENPOSE_OUTPUT, no_display=True, 
             render_pose=0, keypoint_scale=3, 
             frame_rotate=180, face=True, hand=True)
    print(msg)

In [ ]:
#4c. Hand-annotate for face (y/n) on the sample. Save annotations to dataframe.
#TODO: add case to check if annotation column exists, and ask for confirmation to overwrite

annotate_frames(OUTPUT, SAMPLE_JSON_PATH)

In [ ]:
vid_path = '/scratch/users/agrawalk/testvideos/061713-1.AVI'
openpose_vid_output = os.path.join(OPENPOSE_OUTPUT, ntpath.basename(vid_path)[:-4])
p = subprocess.Popen(cmd.format(vid_path, openpose_vid_output), shell=True, 
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(p.communicate()) #Output of job submission command

In [ ]:
#5b. add openpose data to the dataframe.
#TODO: add case to check if openpose column exists, and ask for confirmation to overwrite
#TODO: in this function, create the calculated columns 'face_openpose' and 'face_openpose_body'
incorporate_openpose_output(SAMPLE_JSON_PATH, OPENPOSE_DIR)

In [ ]:
display_prf(SAMPLE_JSON_PATH)

In [ ]:
display_prf2(SAMPLE_JSON_PATH)

In [ ]:
#6. Visualize detector scores

df = pd.read_json('gold_set_sample.json')
df.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#TODO: start using the below:
#from sklearn.metrics import classification_report
#print(classification_report(y_test,predictions))

In [ ]:
X = df.filter(items=[f'face_{det}' for det in ['openpose']])
Y = df['face_present']
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.30, random_state=101)

logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [ ]:
print(len(df[df.index >= 1200]), len(df[df.index < 1200])) #should be 1200 for each
groups = {'random' : df[df.index < 1200], 'face' : df[df.index >= 1200]}

#TODO: dict of dicts is too complicated and doesn't support nice slicing ops,
#switch to using multidimensional array as "map" (where i know that a certain index corresponds to a detector)
#TODO: move this code to helper functions, once polished
#TODO: alternatively, calculate the prf stuff once, save as a json, etc.

avg_metrics = defaultdict(lambda: defaultdict(list))
vid_metrics = defaultdict(lambda: defaultdict(list))
metrics = ['p', 'r', 'f']
det_names = ['mtcnn', 'vj', 'openpose', 'openpose_body', 'pcn']

#Get metrics for each detector
for group in groups:
    for det in det_names:
        cut = groups[group] #slice by face/random
        
        #this is p/r/f for a given det/group
        prf = calc_prf(cut, det) 
        
        #each element is p/r/f for a given det/group/vid
        prf_vids = [calc_prf(cut[cut['vid_name'] == vid_name], det) for vid_name in VID_NAMES] 
        
        for i, metric in enumerate(metrics):
            avg_metrics[metric][det].append(prf[i])
            vid_metrics[metric][det].append([x[i] for x in prf_vids])

print(avg_metrics)
print(vid_metrics)
        

In [ ]:
#Plot metrics
#TODO: move this code to helper functions, once polished

fig, ax = plt.subplots()
index = np.arange(2)
bar_width = 0.15
opacity = 0.8

colors = [f'C{n}' for n in range(10)]

for j, metric in enumerate(metrics):
    plt.figure(j)
    
    #Bar chart plotting
    for i, det in enumerate(det_names):
        x = index + i*bar_width
        plt.bar(x, avg_metrics[metric][det], bar_width,
                alpha=opacity,
                color=colors[i],
                label=det)

    #Line chart plotting
    for i in range(len(VID_NAMES)):
        for k in range(2): #splitting by face/random
            #plot the scores for a given a metric, group, and video across detectors.
            metric_y = [vid_metrics[metric][det][k][i] for det in vid_metrics[metric]]
            metric_x = [index + i*bar_width for i in range(len(det_names))]
            plt.plot(metric_x, metric_y, color='C4', marker='o')
        
    plt.xlabel('Group')
    plt.ylabel('Scores')
    
    metric_print = {'p': 'Precision', 'r': 'Recall', 'f': 'F-score'}
    plt.title(metric_print[metric])
    
    plt.xticks(index + bar_width, ('random', 'face'))
    plt.legend()
    
plt.show()

In [ ]:
df = pd.read_json('gold_set_sample.json')

In [ ]:
df.info()

In [ ]:
openpose_true = df[df['face_present'] == df['face_openpose']]
openpose_false = df[df['face_present'] != df['face_openpose']]


openpose_tp = openpose_true[openpose_true['face_openpose'] == True]
openpose_tn = openpose_true[openpose_true['face_openpose'] == False]

openpose_fp = openpose_false[openpose_false['face_openpose'] == True]
openpose_fn = openpose_false[openpose_false['face_openpose'] == False]

print(len(openpose_tp))
print(len(openpose_tn))
print(len(openpose_fp))
print(len(openpose_fn))

In [ ]:
openpose_tp[-30:]

In [ ]:
openpose_tp[-30:].apply(viz_op_keypoints, axis=1)

In [ ]:
openpose_tp[2:3]
#copy the image
#

In [ ]:
openpose_fp.head()

In [ ]:
openpose_fp[:20].apply(viz_op_keypoints, axis=1)

In [ ]:
openpose_tn[:20].apply(viz_op_keypoints, axis=1)

In [ ]:
openpose_fn[:20].apply(viz_op_keypoints, axis=1)

In [ ]:
#7. Visualize frames which Openpose succeeds (True Positive) and MTCNN fails (False Positive/False Negative).

def openpose_not_mtcnn(row):
    return row['face_present'] == row['face_openpose'] and row['face_present'] != row['face_mtcnn']

op_not_mtcnn = df[df.apply(openpose_not_mtcnn, axis=1)]

mtcnn_fp = op_not_mtcnn[op_not_mtcnn['face_present'] == False] #but MTCNN returned True
mtcnn_fn = op_not_mtcnn[op_not_mtcnn['face_present'] == True] #but MTCNN returned False

print(f'Num False positives: {len(mtcnn_fp)}')
print(f'Num False negatives: {len(mtcnn_fn)}')

In [ ]:
def openpose_succeeds(row):
    return row['face_present'] == row['face_openpose']

op_succeeds = df[df.apply(openpose_succeeds, axis=1)]

In [ ]:
img_paths = [os.path.join(OUTPUT, f'{vid_name}_frames/image-{format_num(num)}.jpg')
                 for vid_name, num in zip(mtcnn_fp['vid_name'], mtcnn_fp['frame'])]
imgs = [plt.imread(path) for path in img_paths]

for img in imgs[:3]:
    plt.imshow(img)
    plt.show()

In [ ]:
#return x and y arrays for a single frame

def get_op_xy(keypt_lists):
    x = []
    y = []
    for keypt in keypt_lists:
        x.append(keypt[0::3]) 
        y.append(keypt[1::3])
    if x == [] or y == []:
        return [], []
    
    return x[0], y[0]

def get_op_lines(x, y, pairings):
    lines = []
    print(f'length of x: {len(x)}')
    print(f'length of y: {len(y)}')

    for x1, y1 in zip(x, y):
        line = []
        for p1, p2 in pairings:
            print(p1, p2)
            line.append([(x[p1], y[p1]), (x[p2], y[p2])])
        lines.append(line)
    return lines

In [ ]:
face_pairings = [0,1,  1,2,  2,3,  3,4,  4,5,  5,6,  6,7,  7,8,  8,9,  9,10,  10,11,  11,12,  12,13,  13,14,  14,15,  15,16,  17,18,  18,19,  19,20, \
                 20,21,  22,23,  23,24,  24,25,  25,26,  27,28,  28,29,  29,30,  31,32,  32,33,  33,34,  34,35,  36,37,  37,38,  38,39,  39,40,  40,41, \
                 41,36,  42,43,  43,44,  44,45,  45,46,  46,47,  47,42,  48,49,  49,50,  50,51,  51,52,  52,53,  53,54,  54,55,  55,56,  56,57,  57,58, \
                 58,59,  59,48,  60,61,  61,62,  62,63,  63,64,  64,65,  65,66,  66,67,  67,60]
pose_pairings = [1,8,   1,2,   1,5,   2,3,   3,4,   5,6,   6,7,   8,9,   9,10,  10,11, 8,12,  12,13, 13,14,  1,0,   0,15, 15,17,  0,16, 16,18,   2,17,  5,18,   14,19,19,20,14,21, 11,22,22,23,11,24]
face_pairings = [(p1, p2) for (p1, p2) in zip(face_pairings[0::2], face_pairings[1::2])]
pose_pairings = [(p1, p2) for (p1, p2) in zip(pose_pairings[0::2], pose_pairings[1::2])]

print(face_pairings)
print(pose_pairings)

In [ ]:
def viz_op_keypoints(row):
    colors = [f'C{n}' for n in range(10)]
    vid_name, num = row['vid_name'], row['frame']
    img_path = os.path.join(OUTPUT, '{0}_frames/image-{1}.jpg'.format(vid_name, format_num(num)))
    img = plt.imread(img_path)
    
    x_pose, y_pose = get_op_xy(row['pose_keypoints'])
    x_face, y_face = get_op_xy(row['face_keypoints'])
    
#     pose_lines = get_op_lines(x_pose, y_pose, pose_pairings)
#     face_lines = get_op_lines(x_face, y_face, face_pairings)
    
    plt.scatter(np.array(x_pose)*720/640, y_pose)
#     lc = mc.LineCollection(pose_lines, colors=[(1, 0, 0, 1)]*len(pose_lines), linewidths=10)
    plt.scatter(np.array(x_face)*720/640, y_face, c=colors[1])
#     lc = mc.LineCollection(face_lines, colors=[(0, 0, 1, 1)]*len(face_lines), linewidths=10)
    plt.imshow(img)
    plt.show()
    
def viz_mtcnn_keypoints(row):
    vid_name, num = row['vid_name'], row['frame']
    img_path = os.path.join(OUTPUT, '{0}_frames/image-{1}.jpg'.format(vid_name, format_num(num)))
    img = plt.imread(img_path)
    print(img.shape)
    

#     x_pose, y_pose = get_op_xy(row['pose_keypoints'])
#     x_face, y_face = get_op_xy(row['face_keypoints'])
    
#     pose_lines = get_op_lines(x_pose, y_pose, pose_pairings)
#     face_lines = get_op_lines(x_face, y_face, face_pairings)
    
#     plt.scatter(np.array(x_pose) + 40, y_pose)
#     lc = mc.LineCollection(pose_lines, colors=[(1, 0, 0, 1)]*len(pose_lines), linewidths=10)
#     plt.scatter(np.array(x_face) + 40, y_face, c=colors[1])
#     lc = mc.LineCollection(face_lines, colors=[(0, 0, 1, 1)]*len(face_lines), linewidths=10)
#     fig,ax = plt.subplots(1)

    # Display the image
    ax.imshow(img)
#     plt.imshow(img)
    for bb in row['bb_mtcnn']:
        x, y, w, h = bb
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    plt.show()
    
# img_paths = [os.path.join(OUTPUT, '{0}_frames/image-{1}.jpg'.format(vid_name, format_num(num)))
#                  for vid_name, num in zip(mtcnn_fn['vid_name'], mtcnn_fn['frame'])][:3]
# imgs = [plt.imread(path) for path in img_paths]


# mtcnn_fn.apply(viz_op_keypoints, axis=1)